In [1]:
import importlib
import torch
import utility
from utility import LeNet
from autoattack_dev import AutoAttack

Re-run the following code to re-import the utility importlib.reload(utility)

In [2]:
net = utility.network()
net.cuda()

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [3]:
orignal_samples = utility.samples(adversarial=False)
utility.evaluate(net, orignal_samples, defense=utility.trimmer, device=torch.device("cuda"))

Check accuracy of the model: 0.98


In [4]:
import torch.nn as nn

class LeNet_BPDA(torch.nn.Module):
    def __init__(self, pretrained_model, defense):
        super(LeNet_BPDA, self).__init__()
        self.pretrained_model = pretrained_model
        self.defense = defense
    
    def forward(self, x):
        defense_val = self.defense(x)
        x_rec = x.clone()
        x_rec.data = defense_val.data
        out = self.pretrained_model(x_rec)
        return out

net_bpda = LeNet_BPDA(net, utility.trimmer)
net_bpda.cuda()

LeNet_BPDA(
  (pretrained_model): LeNet(
    (conv1): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (conv2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (out): Linear(in_features=1568, out_features=10, bias=True)
  )
)

Step 1: set-up BPDA attacks 

In [5]:
inf_BPDA = AutoAttack(net_bpda, norm='Linf', eps=0.3, version='standard')
inf_BPDA.attacks_to_run = ['apgd-ce']
inf_BPDA.apgd.n_restarts = 1
inf_BPDA.apgd.n_iter = 100

setting parameters for standard version


Step 2: apply the APGD

In [7]:
for images, labels in orignal_samples:
    adv_complete = inf_BPDA.run_standard_evaluation(images, labels, bs=100)

using standard version including apgd-ce
initial accuracy: 98.00%
apgd-ce - 1/1 - 93 out of 98 successfully perturbed
robust accuracy after APGD-CE: 5.00% (total time 0.4 s)
max Linf perturbation: 0.30000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 5.00%


In [8]:
BPDA_sample = utility.samples(x=adv_complete)

In [9]:
utility.evaluate(net, BPDA_sample, device=torch.device("cuda"))

Check accuracy of the model: 0.06


In [12]:
utility.evaluate(net_bpda, BPDA_sample, device=torch.device("cuda"))

Check accuracy of the model: 0.05
